In [1]:
import pandas as pd
import numpy as np
import pymongo
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [29]:
ds = pd.read_excel('DBRaw.xlsx')

In [54]:
client = pymongo.MongoClient(os.getenv("URI"))

In [59]:
db = client["Nefro"]
collection = db["exams"]
collection.find_one({"codigo": "33400135"})["exams"]

[{'name': 'Índice de Massa Corporal',
  'data': [{'x': datetime.datetime(2014, 9, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2014, 10, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2014, 11, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2014, 12, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 1, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 2, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 3, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 4, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 5, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 6, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 7, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 8, 1, 0, 0), 'y': 19.3},
   {'x': datetime.datetime(2015, 9, 1, 0, 0), 'y': 19.3},
   {'x': datetime.datetime(2015, 10, 1, 0, 0), 'y': 19.3},
   {'x': datetime.datetime(2015, 11, 1, 0, 0), 'y': 19.3}]},
 {'name': 'Anti-HBs (Titulação)',
  'data': [{'x': datetime.datetime(2014, 9,

In [60]:
pacients_exams = collection.find({"dataset":"DBCleanPlusCompleted2"})

In [61]:

columns = ['Índice de Massa Corporal', 'Anti-HBs (Titulação)', 'Anti-HBs',
       'HBs Ag', 'Anti-HIV', 'Taxa de Redução da Uréia', 'K+', 'Na+',
       'Hb do mês atual', 'Ferritina', 'Saturação de Transferrina',
       'Alb.\n(Verde bromocresol)', 'Proteina Total', 'Colesterol Total',
       'Ca Total do mês atual', 'P', 'Último resultado do PTHi',
       'Fosfatase Alcalina Total']

for p in pacients_exams:
    pacient = ds[ds["Código"] == p["codigo"]]
    pacient = pacient.drop(columns=['Unnamed: 0'])
    pacient = pacient.sort_values(by=['Mês/ano'])
    pacient["Mês/ano"] = pd.to_datetime(pacient["Mês/ano"])

    export = []
    for c in columns:
        if c == "Anti-HBs" or c == "HBs Ag" or c == "Anti-HIV":
                pd.set_option('future.no_silent_downcasting', True)
                pacient[c] = pacient[c].replace('Positivo', 1)
                pacient[c] = pacient[c].replace('Negativo', 0)
                
        
        aux = {"name": (c + " real")}
        res = pacient.loc[(pacient[c].notna()), [c, "Mês/ano"]]
        aux["data"] = [{'x': row['Mês/ano'], 'y': row[c]} for _, row in res.iterrows()]
        
        export.append(aux)
    
    ok = collection.update_one(
        {"codigo": p["codigo"], "dataset": p["dataset"]},
        {"$set": {"exams_real": export}},
        upsert=False
    )

    print(p["codigo"], ok.modified_count)



33430487 1
33430993 1
33431019 1
33431037 1
33431042 1
33431090 1
33431101 1
33431144 1
33431225 1
33431253 1
33431282 1
33431296 1
33431312 1
33431321 1
33431324 1
33431325 1
33431327 1
33440588 1
33440617 1
33440629 1
33440630 1
33440640 1
33440642 1
33440649 1
33440659 1
33440661 1
33440664 1
33440672 1
33440679 1
33440685 1
33440696 1
33440704 1
33440710 1
33440717 1
33440727 1
33450393 1
33450398 1
33461440 1
33461885 1
33461934 1
33462024 1
33462057 1
33462095 1
33462107 1
33462112 1
33462125 1
33462134 1
33462136 1
33462137 1
33462140 1
33462148 1
33462151 1
33462159 1
33462266 1
33470005 1
33471066 1
33471280 1
33471348 1
33471822 1
33471927 1
33471929 1
33471933 1
33471968 1
33471979 1
33471994 1
33472046 1
33472053 1
33472055 1
33472061 1
33472096 1
33472130 1
33472136 1
33472146 1
33500394 1
33500398 1
33500499 1
33500531 1
33500643 1
33500644 1
33500655 1
33500688 1
33500767 1
33510176 1
33510778 1
33510860 1
33520825 1
33520838 1
33520952 1
33521021 1
33521327 1
33521407 1